In [1]:
!pip install -q transformers
!pip install -q tensorflow

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 14.3 MB/s 
     |████████████████████████████████| 59 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 42.5 MB/s 
     |████████████████████████████████| 895 kB 32.0 MB/s 


In [ ]:

from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import tensorflow as tf
import tensorflow_datasets as tfds

import ipywidgets as widgets
from transformers import pipeline
import json
import tweepy
from tweepy import OAuthHandler
import glob
import pandas as pd
import numpy as np
import os
import re
from google.colab import files
from google.colab import drive
drive.mount('/content/drive' ,force_remount=True)
os.listdir()

In [ ]:
os.chdir("drive/MyDrive/vaccine/shuffled2/trimmed json files/")

# data = json.load(open("combined.csv", "r"))

# 

In [ ]:
trimmed_df = pd.read_csv('trimmed_190407_190413.csv', na_filter = False)
trimmed_text = trimmed_df['text'].copy()
trimmed_hashtags = trimmed_df['hashtags'].copy

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-cased"
batch_size = 16    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
token_vectors =tokenizer(list(trimmed_text[0:16]))

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(trimmed_text[0]))

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
max_len = 160
train_input = bert_encode(trimmed_text, tokenizer, max_len=max_len)

In [ ]:
train_input[0]

array([[  101,   155,  1942, ...,     0,     0,     0],
       [  101,   155,  1942, ...,     0,     0,     0],
       [  101,   155,  1942, ...,     0,     0,     0],
       ...,
       [  101, 21534,   146, ...,     0,     0,     0],
       [  101,  2066,  1166, ...,     0,     0,     0],
       [  101,   137,  9300, ...,     0,     0,     0]])

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(train_input[0])
cluster_assignment = clustering_model.labels_

In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(trimmed_text[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
def tokenize_tweet(tweet):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet))

In [ ]:
tokenized_tweets = [tokenize_tweet(tweet) for tweet in trimmed_text]

In [ ]:
type(tokenized_tweets)

list

In [ ]:
tweet_with_len = [[tweet, len(tweet)]
                 for i, tweet in enumerate(tokenized_tweets)]

In [ ]:
tweet_with_len.sort(key=lambda x: x[1])

In [ ]:
import math

In [ ]:
sorted_tweets = [(tweet_text[0] ) for tweet_text in tweet_with_len]

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_tweets, output_types=(tf.int32))

In [ ]:
next(iter(processed_dataset))

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([100], dtype=int32)>

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, )))

In [ ]:
TOTAL_BATCHES = math.ceil(len(sorted_tweets) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 2
X_valid = batched_dataset.take(TEST_BATCHES)
batched_dataset.shuffle(TOTAL_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [ ]:
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

stacked_encoder = keras.models.Sequential([
    keras.layers.Input(shape=(32,)),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu"),
])
stacked_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(154, activation="sigmoid")#,
    # keras.layers.([1, 154])
])
stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])
stacked_ae.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=1.5))
history = stacked_ae.fit(train_data, epochs=20,
                         validation_data=(X_valid))

Epoch 1/20


TypeError: ignored